In [ ]:
# 🔎 1. Importer les librairies nécessaires
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# 📥 2. Charger les données
try:
    df = pd.read_csv('/content/source.csv')
    print("✅ Données chargées avec succès.")
except Exception as e:
    print("❌ Erreur lors du chargement du fichier :", e)

# 📉 3. Définir les features et la target (modèle très simple)
try:
    X = df[['Prix']]  # Uniquement le prix
    y = df['Nom']
except KeyError as e:
    print("❌ Colonne manquante :", e)

# 🔀 4. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 🌲 5. Entraînement du modèle Random Forest
try:
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    print("✅ Random Forest entraîné.")
except Exception as e:
    print("❌ Erreur lors de l'entraînement :", e)

# 📈 6. Prédictions + Accuracy
try:
    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"🌲 Accuracy Random Forest (Prix seul) : {accuracy:.2f}")
except Exception as e:
    print("❌ Erreur de prédiction :", e)


✅ Données chargées avec succès.
✅ Random Forest entraîné.
🌲 Accuracy Random Forest (Prix seul) : 0.41


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 1. Charger le fichier
url = '/content/source.csv'  # upload ce fichier dans Colab
df = pd.read_csv(url)

# 2. Encodage des colonnes catégorielles
le_marque = LabelEncoder()
le_taille = LabelEncoder()
le_nom = LabelEncoder()

df['Marque_enc'] = le_marque.fit_transform(df['Marque'])
df['Taille_enc'] = le_taille.fit_transform(df['Taille'])
df['Nom_enc'] = le_nom.fit_transform(df['Nom'])

# 3. Features et target
X = df[['Marque_enc', 'Taille_enc', 'Prix', 'Categorie']]
y = df['Nom_enc']

# 4. Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Modèle
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 6. Prédictions et accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy : {accuracy:.2f}")

# 7. Exemple de prédiction
def predire_nom(marque, taille, prix, categorie):
    marque_enc = le_marque.transform([marque])[0]
    taille_enc = le_taille.transform([taille])[0]
    features = [[marque_enc, taille_enc, prix, categorie]]
    pred_encoded = model.predict(features)[0]
    return le_nom.inverse_transform([pred_encoded])[0]

# Exemple
print("Exemple : ", predire_nom("DÉLICE", "La bouteille de 1L", 2.5, 1))


Accuracy : 0.80
Exemple :  Jus orange DÉLICE


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# Exemple
print("Exemple : ", predire_nom("BOUCHERIE", "Barquette 250g", 6.8, 3))

Exemple :  Viande hachée 5%


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [1]:
# 1. Installer les librairies nécessaires (RandomForest est inclus dans sklearn)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# 2. Charger le fichier
df = pd.read_csv('/content/source.csv')

# 3. Encodage des colonnes catégorielles
le_marque = LabelEncoder()
le_taille = LabelEncoder()
le_nom = LabelEncoder()

df['Marque_enc'] = le_marque.fit_transform(df['Marque'])
df['Taille_enc'] = le_taille.fit_transform(df['Taille'])
df['Nom_enc'] = le_nom.fit_transform(df['Nom'])

# 4. Définir features et target
X = df[['Marque_enc', 'Taille_enc', 'Prix', 'Categorie']]
y = df['Nom_enc']

# 5. Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 6. Modèle Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 7. Accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Accuracy Random Forest : {accuracy:.2f}")

# 8. Fonction de prédiction (Top 3 sans doublons d'entreprise)
def top3_produits(marque, taille, categorie, prix_min, prix_max):
    subset = df[
        (df['Marque'] == marque) &
        (df['Taille'] == taille) &
        (df['Categorie'] == categorie) &
        (df['Prix'] >= prix_min) &
        (df['Prix'] <= prix_max)
    ]

    if subset.empty:
        return ["❌ Aucun produit trouvé avec ces critères."]

    subset = subset.copy()
    subset['Marque_enc'] = le_marque.transform(subset['Marque'])
    subset['Taille_enc'] = le_taille.transform(subset['Taille'])
    X_subset = subset[['Marque_enc', 'Taille_enc', 'Prix', 'Categorie']]

    # Prédictions
    preds = model.predict(X_subset)
    subset['Prediction'] = preds
    subset['Nom'] = le_nom.inverse_transform(subset['Prediction'])

    # Garder la version la moins chère de chaque entreprise
    subset_unique = (
        subset.sort_values(by='Prix')
        .drop_duplicates(subset=['Entreprise'])
    )

    # Garder les 3 options les moins chères
    final_rows = subset_unique.sort_values(by='Prix').head(3)

    # Formatage des résultats
    resultats = [
        f"{row['Nom']} 🏢 {row['Entreprise']} 💰 {row['Prix']:.2f} DT"
        for _, row in final_rows.iterrows()
    ]
    return resultats

# 9. Exemple d’utilisation
resultats = top3_produits(
    marque="BOUCHERIE",
    taille="Barquette 250g",
    categorie=3,
    prix_min=5.0,
    prix_max=9.0
)

print("🔮 Top 3 suggestions :")
for res in resultats:
    print("🔹", res)


✅ Accuracy Random Forest : 0.79
🔮 Top 3 suggestions :
🔹 Viande hachée 5% 🏢 Géant 💰 5.80 DT
🔹 Viande hachée 5% 🏢 Carrefour 💰 6.80 DT
🔹 Viande hachée 5% 🏢 MG 💰 6.95 DT


In [2]:
# 9. Exemple d’utilisation
resultats = top3_produits(
    marque="BOUCHERIE",
    taille="Barquette 250g",
    categorie=3,
    prix_min=1.0,
    prix_max=1.5
)

print("🔮 Top 3 suggestions :")
for res in resultats:
    print("🔹", res)


🔮 Top 3 suggestions :
🔹 ❌ Aucun produit trouvé avec ces critères.


In [1]:
# 1. Importer les bibliothèques
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# 2. Charger le fichier
df = pd.read_csv('/content/source.csv')

# 3. Encodage des colonnes
le_marque = LabelEncoder()
le_taille = LabelEncoder()
le_nom = LabelEncoder()

df['Marque_enc'] = le_marque.fit_transform(df['Marque'])
df['Taille_enc'] = le_taille.fit_transform(df['Taille'])
df['Nom_enc'] = le_nom.fit_transform(df['Nom'])

# 4. Features et Target
X = df[['Marque_enc', 'Taille_enc', 'Prix', 'Categorie']]
y = df['Nom_enc']

# 5. Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 6. Modèle Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 7. Accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Accuracy Random Forest : {accuracy:.2f}")

# 8. Fonction intelligente de recommandation
def top3_produits(marque=None, taille=None, categorie=None, prix_min=None, prix_max=None):
    if categorie is None:
        return ["❌ La catégorie est obligatoire."]

    # Filtrer uniquement la catégorie
    subset = df[df['Categorie'] == categorie].copy()
    if subset.empty:
        return ["❌ Aucun produit trouvé dans cette catégorie."]

    # Matching des critères
    subset['marque_match'] = (subset['Marque'] == marque) if marque else False
    subset['taille_match'] = (subset['Taille'] == taille) if taille else False
    subset['prix_match'] = (
        subset['Prix'].between(prix_min, prix_max)
        if prix_min is not None and prix_max is not None else False
    )

    # Score total
    subset['score'] = (
        subset['marque_match'].astype(int) +
        subset['taille_match'].astype(int) +
        subset['prix_match'].astype(int)
    )

    if subset['score'].max() == 0:
        return ["❌ Aucun produit proche trouvé. Essayez avec d'autres critères."]

    # Trier par score et prix
    subset = subset.sort_values(by=['score', 'Prix'], ascending=[False, True])
    subset = subset.drop_duplicates(subset=['Entreprise'])

    # Top 3
    top3 = subset.head(3)

    # Préparation des prédictions
    top3['Marque_enc'] = le_marque.transform(top3['Marque'])
    top3['Taille_enc'] = le_taille.transform(top3['Taille'])
    X_top3 = top3[['Marque_enc', 'Taille_enc', 'Prix', 'Categorie']]
    top3['Prediction'] = model.predict(X_top3)
    top3['Nom'] = le_nom.inverse_transform(top3['Prediction'])

    # Résultats
    resultats = [
        f"{row['Nom']} 🏢 {row['Entreprise']} 💰 {row['Prix']:.2f} DT"
        for _, row in top3.iterrows()
    ]

    return resultats

# 9. Exemple d’utilisation
resultats = top3_produits(
    marque="BOUCHERIE",
    taille=None,
    categorie=3,
    prix_min=5.0,
    prix_max=9.0
)

print("🔮 Top 3 suggestions :")
for res in resultats:
    print("🔹", res)


✅ Accuracy Random Forest : 0.79
🔮 Top 3 suggestions :
🔹 Viande hachée 5% 🏢 Géant 💰 5.80 DT
🔹 Viande hachée 5% 🏢 Carrefour 💰 6.80 DT
🔹 Viande hachée 5% 🏢 MG 💰 6.95 DT
